In [1]:
! python -V

Python 3.12.1


In [2]:
!pip install lightgbm
!pip install xgboost
!pip install mlflow


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import mlflow
import pickle
import os

import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.svm import SVC

from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
import lightgbm as lgb

# os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

# TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
# mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

## Preprocessing

In [4]:
def read2df(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df.columns = df.columns.str.lower().str.replace(' ', '_')

    categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

    for c in categorical_columns:
        df[c] = df[c].str.lower().str.replace(' ', '_')

    df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
    df.totalcharges = df.totalcharges.fillna(0)

    df.churn = (df.churn == 'yes').astype(int)

    categorical = [
        'gender',
        'seniorcitizen',
        'partner',
        'dependents',
        'phoneservice',
        'multiplelines',
        'internetservice',
        'onlinesecurity',
        'onlinebackup',
        'deviceprotection',
        'techsupport',
        'streamingtv',
        'streamingmovies',
        'contract',
        'paperlessbilling',
        'paymentmethod',
    ]

    df[categorical] = df[categorical].astype(str) # DictVectorizer inputs dict of strs and outputs numpy array
    
    return df

In [5]:
df = read2df('/workspaces/Churn-Prediction-AWS-Lambda-Deployment/Telco-Customer-Churn.csv')
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


In [6]:
X = df.drop('churn',axis=1)
y = df['churn'].values

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
numerical = ['tenure',
             'monthlycharges',
             'totalcharges'
]

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [9]:
pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

In [11]:
X_train_dict = X_train[categorical + numerical].to_dict(orient='records')
X_val_dict = X_val[categorical + numerical].to_dict(orient='records')

pipeline.fit(X_train_dict, y_train)

,steps,"[('dictvectorizer', ...), ('logisticregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,True
,sort,True
,penalty,'l2'
,dual,False
,tol,0.0001


In [ ]:
# mlflow.set_experiment("exp_1_linear_models")

# mlflow.sklearn.autolog()

# with mlflow.start_run():

#     mlflow.set_tag('developer','nathaniel')


#     mlmodel = LogisticRegression()
#     mlmodel.fit(X_train, y_train)  

#     y_pred = mlmodel.predict(X_val)
#     rmse = root_mean_squared_error(y_val, y_pred)
#     mlflow.log_metric("rmse", rmse)

#     mlflow.sklearn.log_model(mlmodel, artifact_path="models")
#     print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [12]:
y_pred = pipeline.predict(X_val_dict)
rmse = root_mean_squared_error(y_val, y_pred)
rmse

0.4212253577210482

In [13]:
with open('../log_reg.bin','wb') as f_out:
    pickle.dump((pipeline), f_out)